In [1]:
! pip install openpyxl gensim fasttext lightgbm multipledispatch razdel

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
data = pd.read_excel('/kaggle/input/hacksai-3/dataset.xlsx')

## Группа продукции

In [94]:
product_groups = sum(data['Группа продукции'].fillna('')\
                                 .str.strip()\
                                 .str.split('; ')\
                                 .apply(lambda x: [xi.lower().strip() for xi in x]).values, [])

In [102]:
len(set(product_groups))

In [253]:
data[data['Группа продукции'].fillna('').str.split(';').apply(len)>1].shape

In [254]:
data[data['Группа продукции'].fillna('').str.split(';').apply(len)==1].shape

## Коды ТН ВЭД ЕАЭС

In [82]:
data['Коды ТН ВЭД ЕАЭС'].isnull().sum()

In [84]:
data['Коды ТН ВЭД ЕАЭС'].shape

In [85]:
data['Коды ТН ВЭД ЕАЭС'][0]

In [13]:
data['Коды ТН ВЭД ЕАЭС2'] = data['Коды ТН ВЭД ЕАЭС'].fillna('').astype(str).str.split("; ").apply(lambda x: list(set(x)))

In [14]:
ved_from_dataset = sum(data['Коды ТН ВЭД ЕАЭС2'].tolist(), [])

In [15]:
len(set(ved_from_dataset))

## Словарь ВЭД

In [39]:
with open('/kaggle/input/hacksai-3/ved/TNVED/TNVED4.Txt','r', encoding='cp866') as f:
    ved = f.read().split('\n')[1:-1]
    
ved4 = pd.DataFrame([line.split('|')[:-1] for line in ved])
ved4.columns = ['GRUPPA', 'TOV_POZ','SUB_POZ', 'NAIM4','DATA', 'DATA1']


with open('/kaggle/input/hacksai-3/ved/TNVED/TNVED3.Txt','r', encoding='cp866') as f:
    ved = f.read().split('\n')[1:-1]
    
ved3 = pd.DataFrame([line.split('|')[:-1] for line in ved])
ved3.columns = ['GRUPPA', 'TOV_POZ','NAIM3','DATA', 'DATA1']


with open('/kaggle/input/hacksai-3/ved/TNVED/TNVED2.Txt','r', encoding='cp866') as f:
    ved = f.read().split('\n')[1:-1]
    
ved2 = pd.DataFrame([line.split('|')[:-1] for line in ved])
ved2.columns = ['RAZDEL', 'GRUPPA','NAIM2', 'PRIM', 'DATA', 'DATA1']

with open('/kaggle/input/hacksai-3/ved/TNVED/TNVED1.Txt','r', encoding='cp866') as f:
    ved = f.read().split('\n')[1:-1]
    
ved1 = pd.DataFrame([line.split('|')[:-1] for line in ved])
ved1.columns = ['RAZDEL','NAIM1', 'PRIM', 'DATA', 'DATA1']


ved1 = ved1[['RAZDEL', 'NAIM1']].drop_duplicates()
ved2 = ved2[['RAZDEL', 'GRUPPA', 'NAIM2']].drop_duplicates()
ved3 = ved3[['GRUPPA','TOV_POZ', 'NAIM3']].drop_duplicates()
ved4 = ved4[['GRUPPA','TOV_POZ', 'SUB_POZ', 'NAIM4']].drop_duplicates()

ved_final = pd.merge(
            pd.merge(
            pd.merge(ved1, ved2, how='outer'),
                     ved3, how='outer'),
                     ved4, how='outer').dropna(subset=['GRUPPA']).fillna('')

ved_final = pd.concat([ved_final, ved2,ved3, ved4]).fillna('')

In [103]:
ved_final['GRUPPA'].drop_duplicates().shape, ved_final['TOV_POZ'].drop_duplicates().shape, ved_final['SUB_POZ'].drop_duplicates().shape

In [51]:
ved_final['VED'] = ved_final['GRUPPA'] + ved_final['TOV_POZ'] + ved_final['SUB_POZ']

In [52]:
ved_final = ved_final.sort_values(by=['NAIM4', 'NAIM3', 'NAIM2', 'NAIM1'], ascending=False).drop_duplicates(subset='VED')

In [53]:
columns = ['NAIM1', 'NAIM2', 'NAIM3', 'NAIM4']
for col in columns:
    ved_final[col] = ved_final[col].str.replace('-|\xa0|<|>|13', ' ').str.strip().str.lower().str.replace('\s+', ' ')

In [75]:
ved_final.dropna(subset=['VED']).to_csv('ved_dict.csv', sep=';', index=None)

Пересечение

In [63]:
no_match = set(ved_from_dataset) - set(ved_final['VED'].tolist())

In [104]:
len(no_match)

## Регламенты

In [111]:
data['Технические регламенты'].drop_duplicates().values

In [118]:
reglaments = sum(data['Технические регламенты'].fillna('')\
                                 .str.strip()\
                                 .str.split('; ')\
                                 .apply(lambda x: [xi.strip() for xi in x]).values, [])

In [119]:
sorted(set(reglaments))

## Общее наименование продукции

In [138]:
data['Общее наименование продукции'].str.len().max(), data['Общее наименование продукции'].str.len().min()

In [141]:
data['Общее наименование продукции'].str.len()

In [131]:
data[data['Общее наименование продукции'].str.len()<10]['Общее наименование продукции'].value_counts()

In [132]:
data[data['Общее наименование продукции'].str.len()>100]['Общее наименование продукции'].value_counts()